### Извлечение информации для каждого пользователя

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import multiprocessing
import pymorphy2
from concurrent.futures import ThreadPoolExecutor
from itertools import chain
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
num_cores = multiprocessing.cpu_count()

## Функция предобработки названия товара:
- Удаляем обычные цифры и различные скобки
- Приведение к нижнему регистру
- Токенизация(извлечение слов из текста)
- Морфологизация слов(приведение к нормальной форме)

In [2]:
morph = pymorphy2.MorphAnalyzer()

# Загрузка стоп-слов заранее, чтобы не загружать их при каждом вызове функции
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9]', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация текста
    tokens = word_tokenize(text)
    
    preprocessed_text = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]
    
    return preprocessed_text

## Чтение данного нам датасета
1. превращаем имена покупателей в уникальные идентификаторы
2. Извлекаю максимум информации из датасета

In [3]:
df = pd.read_csv('../final_ds_finnopolis.csv', names=['Answer', 'Company_id', 'Price', 'Name'])

In [4]:
df

,Answer,Company_id,Price,Name
0,Оперативная память apacer nox [ah4u16g32c28ymb...,1838685550,4399,Блахов Василий Александрович
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,Блахов Василий Александрович
2,Оперативная память kingston fury beast black r...,1838685550,3599,Блахов Василий Александрович
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,Блахов Василий Александрович
4,Газовые упоры капота geely coolray / джили кул...,1197983357,678,Блахов Василий Александрович
...,...,...,...,...
299995,Колбаса вареная ближние горки молочная гост 450 г,1363498391,369,Малименкова Зоя Александровна
299996,Мед берестов а.с. с прополисом 500 г,1363498391,38,Малименкова Зоя Александровна
299997,Чай черный greenfield magic yunnan в пакетиках...,1976872004,131,Малименкова Зоя Александровна
299998,Колбаса останкино сервелат зернистый варено-ко...,1363498391,329,Малименкова Зоя Александровна


In [5]:
labelencoder = LabelEncoder()
df['Name'] = labelencoder.fit_transform(df.Name)

answerList_for_id - список вещей, которые запросил определенный пользователь  
PriceCount_for_id - Сумма, потраченная определенным человеком  
Company_for_id - сколько потратил определенный человек на определенную компанию

In [6]:
answerList_for_id = df.groupby('Name')['Answer'].agg(list)
PriceCount_for_id = df.groupby('Name')['Price'].agg(sum)
Company_for_id = df.groupby(['Name', 'Company_id'])['Price'].sum()

In [7]:
df

,Answer,Company_id,Price,Name
0,Оперативная память apacer nox [ah4u16g32c28ymb...,1838685550,4399,911
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,911
2,Оперативная память kingston fury beast black r...,1838685550,3599,911
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,911
4,Газовые упоры капота geely coolray / джили кул...,1197983357,678,911
...,...,...,...,...
299995,Колбаса вареная ближние горки молочная гост 450 г,1363498391,369,5336
299996,Мед берестов а.с. с прополисом 500 г,1363498391,38,5336
299997,Чай черный greenfield magic yunnan в пакетиках...,1976872004,131,5336
299998,Колбаса останкино сервелат зернистый варено-ко...,1363498391,329,5336


## Найдем уникальные колонки - виды товаров
##### P.S. датасеты теже - из ozon

In [8]:
onlyfiles = ['../dat2/' + f for f in os.listdir('../dat2') if os.path.isfile(os.path.join('../dat2', f))]

In [9]:
def process_dataframe(file):
    df = pd.read_excel(file)
    df['Категория 1 уровня'] = df['Категория 1 уровня'].astype(str)
    return df['Категория 1 уровня'].unique()
    

col = []
pool = multiprocessing.Pool(processes=num_cores)

# Использование map для параллельной обработки данных
col = pool.map(process_dataframe, onlyfiles)

# Закрытие пула процессов
pool.close()
pool.join()
col = list(set(chain.from_iterable(col)))
col.remove('i9410865678000 _x000D_\n')

Добавим эти колонки в датасет(дублями)
###### Первые нужны для умной системы, вторые для статистики пользователя

In [11]:
for i in col:
    df[i] = [None] * len(df)
for i in col:
    df[i+'_real'] = [None] * len(df)

In [14]:
data = df.iloc[:, [0]+list(range(3, 52))].copy()

In [15]:
data

,Answer,Name,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,...,Товары для мам и детей_real,Одежда_real,Товары повседневного спроса_real,Продукты питания_real,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real
0,Оперативная память apacer nox [ah4u16g32c28ymb...,911,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,911,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Оперативная память kingston fury beast black r...,911,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,911,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Газовые упоры капота geely coolray / джили кул...,911,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Колбаса вареная ближние горки молочная гост 450 г,5336,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299996,Мед берестов а.с. с прополисом 500 г,5336,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299997,Чай черный greenfield magic yunnan в пакетиках...,5336,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299998,Колбаса останкино сервелат зернистый варено-ко...,5336,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Дальше - фишка модели:
- Модель обучается на каждом новом чеке, те она будет все лучше и лучше классифицировать(чтобы не переобучилась стоит низкий learning rate)
- Модель нужна для того чтобы классифицировать название по категориям
- В чем особенность? Благодаря тому, что данная модель - NLP, она способна не только классифицировать, но и искать связи между словами, т.е. с помощью данной модели можно создавать кэшбек для человека так, чтобы он сам того не подозревая хотел именно такой кэшбек. Да и впридачу модель может даже искать товары, которые могут его заинтересовать...
- Достаточно высокая скорость работы и небольшая занимаемая память
- Также преимуществом является то, что для чеков не потребуется хранить информацию по категории, ведь она автоматически определяется с помощью модели. В наше время больших данных это значительно сохраняет память, ведь один пользователь - ничего страшного(size(string)), а если миллиарды человек, которые совершают миллиарды покупок, то получается огромное кол-во занятой памяти

In [17]:
num_cores = multiprocessing.cpu_count()

# Создание пула процессов
pool = multiprocessing.Pool(processes=num_cores)

# Применение функции preprocess_text к столбцу 'Answer'
data['Answer'] = pool.map(preprocess_text, data['Answer'])

# Закрытие пула процессов
pool.close()
pool.join()

In [18]:
data_Ans = data.Answer.tolist().copy()

### Тренируем модель на новых данных

In [19]:
model = Word2Vec.load('word2vec')
model.build_vocab(data_Ans, update=True)
model.alpha = 0.01
model.train(data_Ans, total_examples=model.corpus_count, epochs=1)

Посмотрим пример работы дообученной модели
###### Ксожалению, слова оперативный и одежда далеки по смыслу...

In [20]:
vector1 = model.wv['оперативный']  # Получение векторного представления строки 1
vector2 = model.wv['одежда']  # Получение векторного представления строки 2

# Рассчет косинусного расстояния
model.wv.cosine_similarities(vector1, [vector2])[0]

0.16286726

In [22]:
col_real = [cl + '_real' for cl in col]

### Заполняем колонки категорий:
##### заполняются колонки для рекомендательной системы: выполняется функция relu для каждого элемента и от этого берется среднее. Итог: среднее косинусовое расстояние слов названия от колонки

In [24]:
def calculate_similarity(args):
    index, row, col_name = args
    answer_vectors = [model.wv[word] for word in row['Answer']]
    col_vector = model.wv[col_name]
    similarities = model.wv.cosine_similarities(col_vector, answer_vectors)
    return index, col_name, np.mean(np.maximum(similarities, 0))

with multiprocessing.Pool(processes=num_cores) as pool:
    args_list = ((index, row, col_name) for index, row in data.iterrows() for col_name in data.columns if col_name not in ['Answer', 'Name']+col_real)
    results = pool.imap_unordered(calculate_similarity, args_list)

    for index, col_name, similarity in results:
        data.at[index, col_name] = similarity

In [25]:
data

,Answer,Name,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,...,Товары для мам и детей_real,Одежда_real,Товары повседневного спроса_real,Продукты питания_real,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real
0,"[оперативный, память, apacer, nox, ah4u16g32c2...",911,0.249215,0.305348,0.233985,0.344255,0.376786,0.301514,0.334022,0.359499,...,None,None,None,None,None,None,None,None,None,None
1,"[видеокарта, kfa2, geforce, rtx, x, black, 35n...",911,0.286479,0.425404,0.31453,0.521324,0.44354,0.27,0.2849,0.365769,...,None,None,None,None,None,None,None,None,None,None
2,"[оперативный, память, kingston, fury, beast, b...",911,0.254926,0.30854,0.230929,0.381401,0.364512,0.299406,0.317384,0.374134,...,None,None,None,None,None,None,None,None,None,None
3,"[свеча, зажигание, ngk, cr7hsa, шт]",911,0.192633,0.378024,0.277128,0.370164,0.321038,0.334299,0.180863,0.212025,...,None,None,None,None,None,None,None,None,None,None
4,"[газовый, упор, капот, geely, coolray, джить, ...",911,0.369292,0.448146,0.190871,0.602967,0.495769,0.412625,0.425632,0.287808,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,"[колбаса, варёный, ближний, горка, молочный, г...",5336,0.378298,0.205788,0.229818,0.372594,0.355953,0.232206,0.371526,0.263724,...,None,None,None,None,None,None,None,None,None,None
299996,"[мёд, берест, прополис, г]",5336,0.372194,0.269203,0.170184,0.352033,0.423465,0.210745,0.37829,0.242953,...,None,None,None,None,None,None,None,None,None,None
299997,"[чай, чёрный, greenfield, magic, yunnan, пакет...",5336,0.268559,0.275578,0.386609,0.402984,0.33244,0.23649,0.2799,0.317516,...,None,None,None,None,None,None,None,None,None,None
299998,"[колбаса, останкино, сервелат, зернистый, вари...",5336,0.447397,0.186017,0.202317,0.419306,0.433624,0.231304,0.42041,0.285845,...,None,None,None,None,None,None,None,None,None,None


In [35]:
df2 = data.copy()

In [43]:
df2.iloc[:, 2:]

,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,DIY,NewBorn,...,Товары для мам и детей_real,Одежда_real,Товары повседневного спроса_real,Продукты питания_real,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real
0,0.249215,0.305348,0.233985,0.344255,0.376786,0.301514,0.334022,0.359499,0.224213,0.258979,...,None,None,None,None,None,None,None,None,None,None
1,0.286479,0.425404,0.31453,0.521324,0.44354,0.27,0.2849,0.365769,0.357172,0.232374,...,None,None,None,None,None,None,None,None,None,None
2,0.254926,0.30854,0.230929,0.381401,0.364512,0.299406,0.317384,0.374134,0.264696,0.26616,...,None,None,None,None,None,None,None,None,None,None
3,0.192633,0.378024,0.277128,0.370164,0.321038,0.334299,0.180863,0.212025,0.284665,0.227765,...,None,None,None,None,None,None,None,None,None,None
4,0.369292,0.448146,0.190871,0.602967,0.495769,0.412625,0.425632,0.287808,0.338599,0.130879,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.378298,0.205788,0.229818,0.372594,0.355953,0.232206,0.371526,0.263724,0.26116,0.296103,...,None,None,None,None,None,None,None,None,None,None
299996,0.372194,0.269203,0.170184,0.352033,0.423465,0.210745,0.37829,0.242953,0.149831,0.213824,...,None,None,None,None,None,None,None,None,None,None
299997,0.268559,0.275578,0.386609,0.402984,0.33244,0.23649,0.2799,0.317516,0.231559,0.297035,...,None,None,None,None,None,None,None,None,None,None
299998,0.447397,0.186017,0.202317,0.419306,0.433624,0.231304,0.42041,0.285845,0.331926,0.228734,...,None,None,None,None,None,None,None,None,None,None


#### Приводим колонки к числовому типу

In [49]:
columns_to_convert = df2.columns[2:]
df2[columns_to_convert] = df2[columns_to_convert].astype(float)

In [50]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 50 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Answer                            300000 non-null  object 
 1   Name                              300000 non-null  int64  
 2   Свежие продукты питания           300000 non-null  float64
 3   Ювелирные изделия                 300000 non-null  float64
 4   Товары для дома                   300000 non-null  float64
 5   Entertainment certificates        300000 non-null  float64
 6   Charity                           300000 non-null  float64
 7   Автотовары                        300000 non-null  float64
 8   Крупная бытовая техника           300000 non-null  float64
 9   Товары для животных               300000 non-null  float64
 10  DIY                               300000 non-null  float64
 11  NewBorn                           300000 non-null  f

#### Заполняем колонки для статистики
###### Каждая колонка - кол-во покупок по каждой категории

In [59]:
max_col = df2.drop(['Answer', 'Name'], axis=1).idxmax(axis=1)
result = pd.get_dummies(max_col).add_prefix('Max_')
df2[col_real] = result.replace({False: 0, True: 1})

In [61]:
df2 = df2.groupby('Name').agg(sum)

In [64]:
df2

,Answer,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,DIY,...,Товары повседневного спроса_real,Продукты питания_real,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real,max1_col,max2_col
Name,,,,,,,,,,,,,,,,,,,,,
0,"[технология, проектирование, база, данные, уго...",11.738377,9.366097,8.533210,11.434790,12.639424,8.117991,12.520057,7.260799,8.202822,...,0,0,0,0,0,0,5,0,Charity,Крупная бытовая техника
1,"[крышка, dennerle, nano, cube, стеклянный, бес...",11.149660,8.482357,8.396680,11.661532,12.053114,8.182738,11.368045,8.101865,8.195827,...,0,0,0,3,1,0,3,0,Charity,Entertainment certificates
2,"[python, финансист, тиктокер, работать, китайс...",12.295290,10.055950,7.709215,12.349491,13.077747,8.519625,12.472703,7.998221,7.972781,...,0,0,0,0,0,0,3,0,Charity,Крупная бытовая техника
3,"[свеча, зажигание, ngk, bkr7eix, шт, наконечни...",10.638607,10.078779,7.604977,11.358683,12.445655,7.995968,10.697442,7.232440,8.155302,...,0,0,1,0,0,0,3,0,Charity,Entertainment certificates
4,"[vbs, 1012, щёткодержатель, стартер, audi, a4,...",12.187252,9.018610,8.749717,10.663217,11.870857,8.723648,11.947526,7.173336,7.230455,...,0,0,0,0,0,0,4,0,Свежие продукты питания,Крупная бытовая техника
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,"[интернет, шопинг, реальный, путеводитель, вир...",11.528293,9.367685,7.063229,10.910937,12.830787,7.616346,11.669386,6.897056,7.679736,...,0,0,1,0,0,0,4,0,Charity,Крупная бытовая техника
9968,"[rogue, company, nero, the, great, war, wester...",11.212699,9.712481,8.344623,11.409198,12.929781,7.822716,11.161887,7.631590,7.873966,...,0,0,0,0,0,0,5,0,Charity,Продукты питания
9969,"[фильтр, салонный, marshall, mc1914, lada, ves...",11.692322,9.288435,8.990335,10.923041,13.434611,8.147182,12.319514,7.835769,8.155081,...,0,0,3,1,0,0,4,0,Charity,Крупная бытовая техника


Для умного кэшбэка было бы хорошо найти 3 предсказанных категории

In [66]:
df2['max1_col'] = df2[col].apply(lambda row: row.nlargest(3).index[0], axis=1)
df2['max2_col'] = df2[col].apply(lambda row: row.nlargest(3).index[1], axis=1)
df2['max3_col'] = df2[col].apply(lambda row: row.nlargest(3).index[2], axis=1)

In [67]:
df2

,Answer,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,DIY,...,Продукты питания_real,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real,max1_col,max2_col,max3_col
Name,,,,,,,,,,,,,,,,,,,,,
0,"[технология, проектирование, база, данные, уго...",11.738377,9.366097,8.533210,11.434790,12.639424,8.117991,12.520057,7.260799,8.202822,...,0,0,0,0,0,5,0,Charity,Крупная бытовая техника,Продукты питания
1,"[крышка, dennerle, nano, cube, стеклянный, бес...",11.149660,8.482357,8.396680,11.661532,12.053114,8.182738,11.368045,8.101865,8.195827,...,0,0,3,1,0,3,0,Charity,Entertainment certificates,Крупная бытовая техника
2,"[python, финансист, тиктокер, работать, китайс...",12.295290,10.055950,7.709215,12.349491,13.077747,8.519625,12.472703,7.998221,7.972781,...,0,0,0,0,0,3,0,Charity,Крупная бытовая техника,Entertainment certificates
3,"[свеча, зажигание, ngk, bkr7eix, шт, наконечни...",10.638607,10.078779,7.604977,11.358683,12.445655,7.995968,10.697442,7.232440,8.155302,...,0,1,0,0,0,3,0,Charity,Entertainment certificates,Продукты питания
4,"[vbs, 1012, щёткодержатель, стартер, audi, a4,...",12.187252,9.018610,8.749717,10.663217,11.870857,8.723648,11.947526,7.173336,7.230455,...,0,0,0,0,0,4,0,Свежие продукты питания,Крупная бытовая техника,Charity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,"[интернет, шопинг, реальный, путеводитель, вир...",11.528293,9.367685,7.063229,10.910937,12.830787,7.616346,11.669386,6.897056,7.679736,...,0,1,0,0,0,4,0,Charity,Крупная бытовая техника,Продукты питания
9968,"[rogue, company, nero, the, great, war, wester...",11.212699,9.712481,8.344623,11.409198,12.929781,7.822716,11.161887,7.631590,7.873966,...,0,0,0,0,0,5,0,Charity,Продукты питания,Entertainment certificates
9969,"[фильтр, салонный, marshall, mc1914, lada, ves...",11.692322,9.288435,8.990335,10.923041,13.434611,8.147182,12.319514,7.835769,8.155081,...,0,3,1,0,0,4,0,Charity,Крупная бытовая техника,Продукты питания


Для более подробной информации о пользователе дополним датасет суммарными затратами и потраченных денег на определенную компанию

In [68]:
df2['PriceCount_for_id'] = PriceCount_for_id.values
df2

,Answer,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,DIY,...,Малая бытовая техника_real,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real,max1_col,max2_col,max3_col,PriceCount_for_id
Name,,,,,,,,,,,,,,,,,,,,,
0,"[технология, проектирование, база, данные, уго...",11.738377,9.366097,8.533210,11.434790,12.639424,8.117991,12.520057,7.260799,8.202822,...,0,0,0,0,5,0,Charity,Крупная бытовая техника,Продукты питания,11072
1,"[крышка, dennerle, nano, cube, стеклянный, бес...",11.149660,8.482357,8.396680,11.661532,12.053114,8.182738,11.368045,8.101865,8.195827,...,0,3,1,0,3,0,Charity,Entertainment certificates,Крупная бытовая техника,25740
2,"[python, финансист, тиктокер, работать, китайс...",12.295290,10.055950,7.709215,12.349491,13.077747,8.519625,12.472703,7.998221,7.972781,...,0,0,0,0,3,0,Charity,Крупная бытовая техника,Entertainment certificates,19773
3,"[свеча, зажигание, ngk, bkr7eix, шт, наконечни...",10.638607,10.078779,7.604977,11.358683,12.445655,7.995968,10.697442,7.232440,8.155302,...,1,0,0,0,3,0,Charity,Entertainment certificates,Продукты питания,218323
4,"[vbs, 1012, щёткодержатель, стартер, audi, a4,...",12.187252,9.018610,8.749717,10.663217,11.870857,8.723648,11.947526,7.173336,7.230455,...,0,0,0,0,4,0,Свежие продукты питания,Крупная бытовая техника,Charity,17820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,"[интернет, шопинг, реальный, путеводитель, вир...",11.528293,9.367685,7.063229,10.910937,12.830787,7.616346,11.669386,6.897056,7.679736,...,1,0,0,0,4,0,Charity,Крупная бытовая техника,Продукты питания,77566
9968,"[rogue, company, nero, the, great, war, wester...",11.212699,9.712481,8.344623,11.409198,12.929781,7.822716,11.161887,7.631590,7.873966,...,0,0,0,0,5,0,Charity,Продукты питания,Entertainment certificates,140822
9969,"[фильтр, салонный, marshall, mc1914, lada, ves...",11.692322,9.288435,8.990335,10.923041,13.434611,8.147182,12.319514,7.835769,8.155081,...,3,1,0,0,4,0,Charity,Крупная бытовая техника,Продукты питания,9109


In [69]:
result = df.groupby('Name').apply(lambda group: dict(zip(group['Company_id'], group['Price'])))

df2['PriceToCompany'] = result

### Сохраняем датасет в более презентабельном виде:

In [70]:
df2.index = labelencoder.inverse_transform(df2.iloc[:, -5:].index)

In [78]:
data = df2[df2.columns[1:]].copy()

In [79]:
data.head()

,Свежие продукты питания,Ювелирные изделия,Товары для дома,Entertainment certificates,Charity,Автотовары,Крупная бытовая техника,Товары для животных,DIY,NewBorn,...,Книги_real,ТВ и аудио_real,Аптека_real,Спорт и отдых_real,Красота и здоровье_real,max1_col,max2_col,max3_col,PriceCount_for_id,PriceToCompany
Абакелия Мукас Робертовна,11.738377,9.366097,8.533210,11.434790,12.639424,8.117991,12.520057,7.260799,8.202822,7.748003,...,0,0,0,5,0,Charity,Крупная бытовая техника,Продукты питания,11072,"{1252535439: 989, 1310763416: 498, 1983695308:..."
Абакумов Геннадий Николаевич,11.149660,8.482357,8.396680,11.661532,12.053114,8.182738,11.368045,8.101865,8.195827,7.954131,...,3,1,0,3,0,Charity,Entertainment certificates,Крупная бытовая техника,25740,"{1221453650: 1930, 1816854351: 1583, 169278206..."
Абакумова Анна Ефимовна,12.295290,10.055950,7.709215,12.349491,13.077747,8.519625,12.472703,7.998221,7.972781,7.837850,...,0,0,0,3,0,Charity,Крупная бытовая техника,Entertainment certificates,19773,"{1310763416: 2685, 1252535439: 1359, 139734612..."
Абакумова Евгения Александровна,10.638607,10.078779,7.604977,11.358683,12.445655,7.995968,10.697442,7.232440,8.155302,7.076545,...,0,0,0,3,0,Charity,Entertainment certificates,Продукты питания,218323,"{1085769503: 5580, 1197983357: 273, 1597283624..."
Абалкина Мария Егоровна,12.187252,9.018610,8.749717,10.663217,11.870857,8.723648,11.947526,7.173336,7.230455,8.257886,...,0,0,0,4,0,Свежие продукты питания,Крупная бытовая техника,Charity,17820,"{1085769503: 1360, 1197983357: 508, 1252535439..."


In [80]:
data.to_csv('../final_dataset.csv')